In [35]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly_express as px

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

from preprocessing import *
from feature_engineering import *


device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [36]:
df = pd.read_csv("E-Building_Data.csv")
df['date_time'] = pd.to_datetime(df['date_time'])

In [37]:
fe = FeatureEngineering(df) # helper class for feature enginerring
df = fe.feature_engineering(n=False, categorical_features=["season"])

In [38]:
# lat, long for weather station in karlsruhe
latitude = 49.0069
longitude = 8.4037
start_date = datetime(2022, 6, 2)
end_date = datetime(2023, 9, 30)
wf = WeatherFetcher(latitude, longitude, start_date, end_date) # helper class to fetch weather with api

In [39]:
df = fe.filter_rooms_by_prefix()
prefixes = ['e0', 'e1', 'e2', 'e3']
dataframes = [df[prefix] for prefix in prefixes]


dfe0, dfe1, dfe2, dfe3 = dataframes

In [40]:
dataframes = [dfe0, dfe1, dfe2, dfe3]

for i in range(len(dataframes)):
    dataframes[i] = wf.combine_weather(dataframes[i])
dfe0, dfe1, dfe2, dfe3 = dataframes

def one_hot_encode_room_number(df):
    return pd.get_dummies(df, columns=['room_number'], dtype="int")
encoded_dataframes = [one_hot_encode_room_number(df) for df in dataframes]

dfe0, dfe1, dfe2, dfe3 = encoded_dataframes

In [41]:

def sliding_window_forecast(data: pd.DataFrame, label_name: str, n_in: int, n_out: int, dropna: bool = True) -> pd.DataFrame:
    """
    Transforms time series data into a supervised learning format for forecasting.

    Parameters:
    - data (pd.DataFrame): The input time series data.
    - label_name (str): The name of the target column to forecast.
    - n_in (int): Number of lag observations (input sequence length).
    - n_out (int): Number of future observations to forecast (output sequence length).
    - dropna (bool): Whether to drop rows with NaN values.

    Returns:
    - pd.DataFrame: The transformed DataFrame suitable for forecasting.
    """
    if label_name not in data.columns:
        raise ValueError(f"label_name '{label_name}' is not a column in the data")

    cols, names = list(), list()

    # Input sequence (t-n, ..., t-1, t)
    for i in range(n_in, -1, -1):
        cols.append(data.shift(i))
        names += [f"{col}(t-{i})" if i > 0 else f"{col}(t)" for col in data.columns]

    # Forecast sequence (t+1, ..., t+n)
    for i in range(1, n_out + 1):
        cols.append(data[[label_name]].shift(-i))
        names += [f"{label_name}(t+{i})"]

    
    data_reframed = pd.concat(cols, axis=1)
    data_reframed.columns = names

    if dropna:
        data_reframed.dropna(inplace=True)

    return data_reframed

In [42]:
dataframes = [dfe0, dfe1, dfe2, dfe3]
[df.set_index("date_time", inplace=True) for df in dataframes]

[None, None, None, None]

In [44]:
dataframes = [dfe0, dfe1, dfe2, dfe3]
transformed_dataframes = [sliding_window_forecast(df, "tmp", 1, 1) for df in dataframes] # forecarst the next datapoint with the previous datapoint
dfe0, dfe1, dfe2, dfe3 = transformed_dataframes

In [46]:
def prepare_data(df):
    input_data = df.drop(["tmp(t+1)"], axis=1).values
    targets = df[["tmp(t+1)"]].values
    T = 1  # Number of timesteps to look while predicting
    D = input_data.shape[1]  # Dimensionality of the input
    N = len(input_data) - T

    # Train size: 80% of the total data size
    train_size = int(len(input_data) * 0.80)

    # Normalization of the inputs
    scaler = StandardScaler()
    scaler.fit(input_data[:train_size + T - 1])
    input_data = scaler.transform(input_data)

    # Preparing X_train and y_train
    X_train = np.zeros((train_size, T, D))
    y_train = np.zeros((train_size, 1))

    for t in range(train_size):
        X_train[t, :, :] = input_data[t:t+T]
        y_train[t] = (targets[t+T])

    # Preparing X_test and y_test
    X_test = np.zeros((N - train_size, T, D))
    y_test = np.zeros((N - train_size, 1))

    for i in range(N - train_size):
        t = i + train_size
        X_test[i, :, :] = input_data[t:t+T]
        y_test[i] = (targets[t+T])

    
    X_train = torch.from_numpy(X_train.astype(np.float32))
    y_train = torch.from_numpy(y_train.astype(np.float32))
    X_test = torch.from_numpy(X_test.astype(np.float32))
    y_test = torch.from_numpy(y_test.astype(np.float32))

    return X_train, y_train, X_test, y_test

In [48]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTM, self).__init__()
        self.M = hidden_dim
        self.L = layer_dim

        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=layer_dim,
            batch_first=True)
        
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, X):
        # Check if X is batched or unbatched
        if len(X.shape) == 2:  # Unbatched input: (sequence_length, input_dim)
            X = X.unsqueeze(0)  # Add batch dimension: (1, sequence_length, input_dim)

        batch_size = X.size(0)

        
        h0 = torch.zeros(self.L, batch_size, self.M).to(X.device)  # (num_layers, batch_size, hidden_dim)
        c0 = torch.zeros(self.L, batch_size, self.M).to(X.device)  # (num_layers, batch_size, hidden_dim)

       
        out, (hn, cn) = self.lstm(X, (h0.detach(), c0.detach()))

        
        out = self.dropout(out)

        
        out = self.fc(out[:, -1, :])
        return out


### Uncomment this cell, if u want to train the models takes about 60min for all 4 models.

In [49]:

def train(model, learning_rate, X_train, y_train, X_test, y_test, batch_size=64, epochs=100):

    
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    train_losses = []
    test_losses = []
    
    
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    for epoch in range(epochs):
        model.train()
        
        epoch_train_loss = 0.0
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            epoch_train_loss += loss.item()
        
        # Compute average training loss for the epoch
        train_loss = epoch_train_loss / len(train_loader)
        train_losses.append(train_loss)
        
        # Compute test loss
        model.eval()
        epoch_test_loss = 0.0
        with torch.no_grad():
            for batch_X_test, batch_y_test in test_loader:
                test_outputs = model(batch_X_test)
                loss = criterion(test_outputs, batch_y_test)
                epoch_test_loss += loss.item()
        
        # Compute average test loss for the epoch
        test_loss = epoch_test_loss / len(test_loader)
        test_losses.append(test_loss)
        
        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')
    
    return train_losses, test_losses



floor_dfs = [dfe0, dfe1, dfe2, dfe3]
for i, df in enumerate(floor_dfs, start=0):
    X_train, y_train, X_test, y_test = prepare_data(df)
# Define the LSTM model
    input_size = X_train.shape[2]  # shape of D, input data
    hidden_size = 64
    layer_size = 1
    output_size = 1
    model = LSTM(input_size, hidden_size, layer_size, output_size)
        
    # Train the model for the current floor
    learning_rate = 0.001
    epochs = 100
    batch_size = 32
    train_losses, test_losses = train(model, learning_rate, X_train, y_train, X_test, y_test, epochs=epochs, batch_size=batch_size)
    
    # Save the trained model and losses
    torch.save(model.state_dict(), f'floor{i}_lstm_model.pth')
    with open(f'floor{i}_losses.txt', 'w') as file:
        file.write("Train Losses:\n")
        for loss in train_losses:
            file.write(f"{loss}\n")
        file.write("\nTest Losses:\n")
        for loss in test_losses:
            file.write(f"{loss}\n")
    
    print(f'Model for floor {i} trained, saved, and losses recorded.')

Epoch [1/100], Train Loss: 224.2159, Test Loss: 553.9844
Epoch [10/100], Train Loss: 9.2293, Test Loss: 5.9475
Epoch [20/100], Train Loss: 5.3934, Test Loss: 1.4953
Epoch [30/100], Train Loss: 3.4123, Test Loss: 2.4946
Epoch [40/100], Train Loss: 2.1793, Test Loss: 1.8757
Epoch [50/100], Train Loss: 1.4775, Test Loss: 2.9591
Epoch [60/100], Train Loss: 1.1362, Test Loss: 1.8008
Epoch [70/100], Train Loss: 1.0201, Test Loss: 1.1209
Epoch [80/100], Train Loss: 0.9704, Test Loss: 0.6930
Epoch [90/100], Train Loss: 0.9319, Test Loss: 0.5753
Epoch [100/100], Train Loss: 0.9173, Test Loss: 0.5631
Model for floor 0 trained, saved, and losses recorded.
Epoch [1/100], Train Loss: 223.4818, Test Loss: 635.2549
Epoch [10/100], Train Loss: 8.6403, Test Loss: 2.1615
Epoch [20/100], Train Loss: 4.2930, Test Loss: 1.2933
Epoch [30/100], Train Loss: 2.2859, Test Loss: 1.3212
Epoch [40/100], Train Loss: 1.5573, Test Loss: 2.0034
Epoch [50/100], Train Loss: 1.3967, Test Loss: 1.2555
Epoch [60/100], Trai

In [50]:
# List of dataframes
dataframes = [dfe0, dfe1, dfe2, dfe3]

# Process each dataframe and print the shape
input_data_list = []
for i, df in enumerate(dataframes):
    input_data = df.drop(["tmp(t)"], axis=1)
    input_data_list.append(input_data)
    print(f"Shape of input_data{i+1}: {input_data.shape}") # depending of number of rooms in each floor

Shape of input_data1: (41831, 58)
Shape of input_data2: (54989, 62)
Shape of input_data3: (32908, 58)
Shape of input_data4: (53657, 62)


In [52]:
# Directory where the models are stored
model_dir = "trained_models"

model_details = [
    {"path": "floor0_lstm_model.pth", "input_size": 58},
    {"path": "floor1_lstm_model.pth", "input_size": 62},
    {"path": "floor2_lstm_model.pth", "input_size": 58},
    {"path": "floor3_lstm_model.pth", "input_size": 62},
]


models = []
for details in model_details:
    path = os.path.join(model_dir, details["path"])  # Add directory to the path
    input_size = details["input_size"]
    hidden_size = 64
    num_layers = 1
    output_size = 1

    # Instantiate and load the model
    model = LSTM(input_size, hidden_size, num_layers, output_size)
    model.load_state_dict(torch.load(path))
    model.eval()
    models.append(model)

model0, model1, model2, model3 = models[:4]

In [49]:
for i, df in enumerate(dataframes, start=0):
    # Load losses from the file
    with open(f'floor{i}_losses.txt', 'r') as file:
        lines = file.readlines()
    
    
    train_losses = []
    test_losses = []
    is_test_loss = False
    
    for line in lines:
        if line.strip() == 'Train Losses:':
            is_test_loss = False
            continue
        elif line.strip() == 'Test Losses:':
            is_test_loss = True
            continue
        elif line.strip() == '':
            continue
        
        if not is_test_loss:
            train_losses.append(float(line.strip()))
        else:
            test_losses.append(float(line.strip()))
        
    epochs = range(1, len(train_losses) + 1)
    loss_data = {'Epochs': epochs, 'Train Loss': train_losses, 'Test Loss': test_losses}
    loss_df = pd.DataFrame(loss_data)
    
    # Plot using Plotly Express
    fig = px.line(loss_df, x='Epochs', y=['Train Loss', 'Test Loss'], title=f'Loss for Floor {i}')
    fig.update_layout(yaxis_title="Loss")
    fig.show()

In [62]:
models = [model0, model1, model2, model3]
dataframes = [dfe0, dfe1, dfe2, dfe3]

results = {
    'Floor 0 True Values': [],
    'Floor 1 True Values': [],
    'Floor 2 True Values': [],
    'Floor 3 True Values': [],
    'Floor 0 Predictions': [],
    'Floor 1 Predictions': [],
    'Floor 2 Predictions': [],
    'Floor 3 Predictions': []
}

# Determine device once
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Iterate over each model and dataframe
for model_name, (df, model) in zip(['Floor 0', 'Floor 1', 'Floor 2', 'Floor 3'], zip(dataframes, models)):
    X_train, y_train, X_test, y_test = prepare_data(df)

    # Move model and data to the device
    model.to(device)
    X_test = X_test.to(device)
    y_test = y_test.to(device)

    # Set model to evaluation mode
    model.eval()

    # Perform prediction
    with torch.no_grad():
        y_pred = model(X_test)

    y_pred = y_pred.cpu().numpy()
    y_test = y_test.cpu().numpy()

    # Update the results dictionary
    results[f'{model_name} True Values'].extend(y_test.flatten())
    results[f'{model_name} Predictions'].extend(y_pred.flatten())

# Debug statement to check lengths of arrays in the results dictionary
print("Lengths of arrays in the results dictionary:")
for key, value in results.items():
    print(f"{key}: {len(value)}")

# Ensure all arrays have the same length before creating DataFrame
min_length = min(len(value) for value in results.values())

# Trim all arrays to the same length
for key in results:
    results[key] = results[key][:min_length]

# Convert results dictionary to DataFrame
df_results = pd.DataFrame(results)

Lengths of arrays in the results dictionary:
Floor 0 True Values: 8366
Floor 1 True Values: 10997
Floor 2 True Values: 6581
Floor 3 True Values: 10731
Floor 0 Predictions: 8366
Floor 1 Predictions: 10997
Floor 2 Predictions: 6581
Floor 3 Predictions: 10731


In [63]:
df_results

,Floor 0 True Values,Floor 1 True Values,Floor 2 True Values,Floor 3 True Values,Floor 0 Predictions,Floor 1 Predictions,Floor 2 Predictions,Floor 3 Predictions
0,26.157499,27.628666,27.610001,29.502857,26.329964,27.120182,28.157795,29.068117
1,26.807501,28.129545,28.204000,29.475000,26.346046,27.337040,28.169342,29.034254
2,27.474445,28.642500,27.940001,29.440001,26.110584,27.253504,27.971241,29.385424
3,26.452000,28.245001,27.562424,29.184999,26.286661,27.336596,27.872971,29.014044
4,26.257500,27.959999,28.128666,29.371429,25.919628,27.082561,28.070292,29.059143
...,...,...,...,...,...,...,...,...
6576,24.219999,25.198000,24.690001,25.716000,24.246798,25.670193,24.044195,25.409529
6577,25.432501,26.202499,24.402500,25.174999,24.119560,25.481613,24.638906,25.675598
6578,24.000000,25.440001,24.115715,25.200001,24.497641,25.594536,24.584267,25.587200
6579,24.434999,24.903334,22.728333,25.721428,24.147882,25.919022,25.128349,25.669382


In [68]:

dataframes = [dfe0, dfe1, dfe2, dfe3]
models = [model0, model1, model2, model3]


figures = []

# Iterate over each model and dataframe
for floor, (df, model) in enumerate(zip(dataframes, models)):
    
    X_train, y_train, X_test, y_test = prepare_data(df)

    # Set model to evaluation mode
    model.eval()

    # Perform prediction
    with torch.no_grad():
        if torch.cuda.is_available():
            X_test = X_test.cuda()
        y_pred = model(X_test)

    
    if torch.cuda.is_available():
        y_pred = y_pred.cpu().numpy()
        y_test = y_test.cpu().numpy()
    else:
        y_pred = y_pred.numpy()
        y_test = y_test.numpy()

    # Create a DataFrame with true values and predicted values
    df_results = pd.DataFrame({
        'Index': range(len(y_test)),
        'True Values': y_test.flatten(),
        'Predicted Values': y_pred.flatten()
    })

    
    df_melted = df_results.melt(id_vars=['Index'], value_vars=['True Values', 'Predicted Values'],
                                var_name='Type', value_name='Value')

    
    fig = px.line(df_melted, x='Index', y='Value', color='Type', title=f'Etage {floor}: Vorhersage von t+1 mit t-1 and t')
    figures.append(fig)

for fig in figures:
    fig.show()


In [65]:
from sklearn.metrics import mean_squared_error, mean_absolute_error


# Initialize a dictionary to store performance metrics for each floor
metrics = {}
floor_names = ['Floor 0', 'Floor 1', 'Floor 2', 'Floor 3']

for floor_name in floor_names:
    true_values_col = f'{floor_name} True Values'
    predictions_col = f'{floor_name} Predictions'
    
    if true_values_col not in df_results.columns or predictions_col not in df_results.columns:
        print(f"Error: {true_values_col} or {predictions_col} not found in df_results columns.")
        continue
    
    true_values = df_results[true_values_col]
    predictions = df_results[predictions_col]
    
    mse = mean_squared_error(true_values, predictions)
    mae = mean_absolute_error(true_values, predictions)
    
    metrics[floor_name] = {'MSE': mse, 'MAE': mae}

# Print performance metrics
for floor_name, metric_vals in metrics.items():
    print(f"Floor: {floor_name}")
    print(f"MSE: {metric_vals['MSE']:.4f}")
    print(f"MAE: {metric_vals['MAE']:.4f}")

Floor: Floor 0
MSE: 0.4767
MAE: 0.5400
Floor: Floor 1
MSE: 1.1832
MAE: 0.8671
Floor: Floor 2
MSE: 1.6807
MAE: 1.0433
Floor: Floor 3
MSE: 1.0731
MAE: 0.8424
